# Fragrantica perfume review clasifier (LSTM with stopword removed)

In [1]:
import pandas as pd
import requests
from os import path
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# Model Name
MODEL_NAME = 'lstm_with_lemmatizated_02'

# HyperParameters
PAD_LEN = 200
NUM_WORDS = 2500
EMBEDDING = 100
BATCH_SIZE = 256


In [3]:
data_path = '../dataset/dataset_210626_215600.csv'
data_exist = path.exists(data_path)

if not data_exist:
    url = 'https://kyuuuw-nlp-dataset.s3.ap-northeast-2.amazonaws.com/fragrantica/dataset_210626_215600.csv'
    r = requests.get(url, allow_redirects=True)
    open(data_path, 'w').write(r.text)

In [4]:
data = pd.read_csv(data_path)

X_data = data['lemmatizated']
y_data = data['label']

print(len(X_data))
print(len(y_data))


##### 토큰화 및 인덱스 부여

74779
74779


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(X_data[0])
print(len(sequences[0]))

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
76


##### 인덱스별 단어 표시

In [6]:
word_to_index = tokenizer.word_index

##### 빈도수 분석

In [7]:
threshold = 2
total_cnt = len(word_to_index) # 총 단어의 수
rare_cnt = 0 # 등장 빈도 수가 threshold 보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"등장 빈도가 {threshold}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합(vocabulary)에서 희귀 단어의 비율: {rare_cnt / total_cnt}" )
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {rare_freq / total_freq}")

등장 빈도가 2번 이하인 희귀 단어의 수: 43039
단어 집합(vocabulary)에서 희귀 단어의 비율: 0.5753877005347594
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.012692410682162147


##### 등장 횟수 상위 50000개의 word 만 사용

In [8]:
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(sequences[:5])

[[6, 72, 170, 54, 93, 1439, 14, 1, 25, 1027, 1892, 824, 888, 463, 107, 185, 31, 309, 39, 1190, 25, 2495, 2415, 1, 15, 6, 1472, 50, 80, 2482, 3, 158, 666, 255, 320, 123, 1181, 108, 67, 152, 1439, 621, 31, 263, 191, 4, 1, 1389, 336, 463, 413, 46, 108, 175, 303, 521, 218, 168, 44, 1690, 349, 312, 895, 432, 474, 39, 28, 350, 22, 179, 11], [21, 464, 121, 964, 1, 2, 1209, 25, 2, 10, 769, 494, 64, 8, 1, 659, 61, 188, 73, 33, 64, 37, 731, 8, 163, 8, 349, 1013, 1691, 1019, 492, 10, 52, 54, 802, 11, 1080, 783, 309, 39, 28, 25, 263, 1594, 424, 213, 117, 1140, 61, 10, 152, 775, 9, 24, 357, 1, 1, 152, 1031, 67, 139, 437, 44, 24, 223, 200, 38, 13, 659, 81, 83, 1350, 78, 135, 204, 263, 73, 439, 26, 4, 30, 1, 301, 14, 40, 235, 208, 97, 90, 1611, 8, 8, 254, 223, 1595, 1629, 30, 117, 43, 14, 759, 11, 335, 7, 335, 176, 177, 601, 1101, 22, 11, 243, 15, 121, 219, 13, 823], [10, 365, 63, 238, 129, 10, 138, 52, 32, 2326, 6, 448, 6, 72, 313, 1220, 495, 85, 92, 406, 88, 135, 226, 889, 557, 388, 207, 4, 9, 152,

##### Split data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(sequences, y_data)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

56084
56084
18695
18695


In [10]:
# padding and trimming
X_train = pad_sequences(X_train, maxlen=PAD_LEN)
X_test = pad_sequences(X_test, maxlen=PAD_LEN)

In [11]:
# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[0])

[1. 0. 0. 0.]


In [12]:
model = Sequential()
model.add(Embedding(NUM_WORDS, EMBEDDING))
model.add(LSTM(EMBEDDING))
model.add(Dense(4, activation='softmax'))

model.summary()

[2021-06-26 18:20:49.857 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:319 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-26 18:20:49.881 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:319 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         250000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 330,804
Trainable params: 330,804
Non-trainable params: 0
_________________________________________________________________


In [13]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'../model/{MODEL_NAME}.h5', monitor='val_acc', mode='max')

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [15]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE, epochs=30, callbacks=[es, mc],
                    validation_data=(X_test, y_test))

Epoch 1/30
220/220 [==============================] - 7s 33ms/step - loss: 1.0582 - acc: 0.5493 - val_loss: 0.8300 - val_acc: 0.6814
Epoch 2/30
220/220 [==============================] - 7s 31ms/step - loss: 0.7161 - acc: 0.7307 - val_loss: 0.7477 - val_acc: 0.7159
Epoch 3/30
220/220 [==============================] - 7s 30ms/step - loss: 0.6426 - acc: 0.7574 - val_loss: 0.6893 - val_acc: 0.7379
Epoch 4/30
220/220 [==============================] - 7s 30ms/step - loss: 0.6234 - acc: 0.7653 - val_loss: 0.6797 - val_acc: 0.7384
Epoch 5/30
220/220 [==============================] - 7s 31ms/step - loss: 0.5973 - acc: 0.7734 - val_loss: 0.6866 - val_acc: 0.7358
Epoch 6/30
220/220 [==============================] - 7s 31ms/step - loss: 0.5864 - acc: 0.7752 - val_loss: 0.6972 - val_acc: 0.7363
Epoch 7/30
220/220 [==============================] - 7s 31ms/step - loss: 0.5741 - acc: 0.7823 - val_loss: 0.6919 - val_acc: 0.7351
Epoch 8/30
220/220 [==============================] - 7s 31ms/step - 